# Assignment 3 - Boosting

Freek van Geffen | s2633256 <br>
Justin Kraaijenbrink | s2577984

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, explained_variance_score

import xgboost as xgb
import sklearn.metrics as metrics
import pickle

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib import cm

## Regression

#### Import data and define helper functions

In [3]:
import os
path = os.getcwd()

housing = pd.read_csv(path + '/data/Housing.csv')

In [4]:
# Standardization function to obtain standardized regression coefficients 
# for the linear regression model
def Standardize(model, X_train, y_train, index):
    X_train = X_train.drop(columns = 'const')
    index = index.drop('const')
    
    sdy = np.std(y_train)
    sdx = np.std(X_train[index]).values
    coefs = model.params.drop('const')
    
    std_coefs = coefs.values / (sdy / sdx)
    
    out = pd.DataFrame(std_coefs, index = coefs.index, columns = ['Std. Beta']).sort_values(by = 'Std. Beta', ascending = False)
    
    return(out)

In [5]:
# Function to evaluate the performance of the models
def CalculateRMSE(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    mse = np.sum((y_pred - y_test)**2) / len(y_test)
    
    return np.sqrt(mse)

In [6]:
# Generate train and test set
np.random.RandomState(248)
y = housing['price']
X = housing.iloc[:, 1:]
X = pd.get_dummies(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 248)

#### Boosting

In [ ]:
def PlotTuningResults3D(grid, param_test, parameter1, parameter2):
    m = len(param_test[parameter1])
    n = len(param_test[parameter2])
    
    param_name1 = 'param_' + parameter1
    param_name2 = 'param_' + parameter2
    X = np.reshape(grid.cv_results_[param_name1].data,[n,m])
    Y = np.reshape(grid.cv_results_[param_name2].data,[n,m])
    Z = np.reshape(grid.cv_results_['mean_test_score'],[n,m])
    
    Y = np.array(Y, dtype = float)
    X = np.array(X, dtype = float)
    
    fig = plt.figure(figsize = (12,8))
    ax = fig.gca(projection='3d')

    # Plot the surface.
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    
    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.set_xlabel(parameter1, size = 13)
    ax.set_ylabel(parameter2, size = 13)
    ax.set_zlabel('Test explained variance', size = 13)

    plt.show()

In [ ]:
dtrain = xgb.DMatrix(data = X_train, label=list(y_train.values))

In [ ]:
xgb1 = xgb.XGBRegressor(
    n_estimators=10000,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    n_jobs = -1,
    seed=248)

cvresult = xgb.cv(xgb1.get_xgb_params(), dtrain, num_boost_round=xgb1.n_estimators, nfold=5,
                      metrics='rmse', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
# CV for optimal max depth and child weight
param_test1 = {
 'max_depth':range(1,11,1),
 'min_child_weight':range(0,11,1)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBRegressor(n_estimators=81, 
                                                     objective= 'reg:squarederror',
                                                     tree_method = 'auto',
                                                     seed=248), 
                        param_grid = param_test1, 
                        scoring='explained_variance',
                        n_jobs = -1,
                        cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.best_params_

In [ ]:
PlotTuningResults3D(gsearch1, param_test1, 'max_depth', 'min_child_weight')

In [ ]:
xgb2 = xgb.XGBRegressor(
    n_estimators=10000,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    max_depth = 4,
    min_child_weights = 3,
    n_jobs = -1,
    seed=248)

cvresult = xgb.cv(xgb2.get_xgb_params(), dtrain, num_boost_round=xgb2.n_estimators, nfold=5,
                      metrics='rmse', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test2 = {'min_split_loss':[i/100.0 for i in range(0,2, 1)]}
gsearch2 = GridSearchCV(estimator = xgb.XGBRegressor(n_estimators=55, 
                                                     objective= 'reg:squarederror',
                                                     tree_method = 'hist',
                                                     seed=248,
                                                    max_depth = 4,
                                                    min_child_weights = 3), 
                        param_grid = param_test2, 
                        scoring='explained_variance',
                        n_jobs = -1,
                        cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_

In [ ]:
xgb3 = xgb.XGBRegressor(
    n_estimators=10000,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    max_depth = 4,
    min_child_weights = 3,
    min_split_loss = 0,
    n_jobs = -1,
    seed=248)

cvresult = xgb.cv(xgb3.get_xgb_params(), dtrain, num_boost_round=xgb3.n_estimators, nfold=5,
                      metrics='rmse', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test3 = {'subsample':[i/100.0 for i in range(5,105, 5)],
               'colsample_bytree':[i/100.0 for i in range(0,105, 5)]}

gsearch3 = GridSearchCV(estimator = xgb.XGBRegressor( n_estimators=55, 
                                                  max_depth=4,
                                                  min_child_weight=3,
                                                  min_split_loss = 0,
                                                  objective = 'reg:squarederror',
                                                  tree_method = 'auto',
                                                  seed=248),
                        param_grid = param_test3, 
                        scoring='explained_variance',
                        n_jobs=-1,
                        cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_

In [ ]:
PlotTuningResults3D(gsearch3, param_test3, 'subsample', 'colsample_bytree')

In [ ]:
xgb4 = xgb.XGBRegressor(
    n_estimators=10000,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    max_depth = 3,
    min_child_weights = 0,
    min_split_loss = 0,
    colsample_bytree = 0.9,
    subsample = 0.85,
    n_jobs = -1,
    seed=248)

cvresult = xgb.cv(xgb4.get_xgb_params(), dtrain, num_boost_round=xgb4.n_estimators, nfold=5,
                      metrics='rmse', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test4 = {
 'reg_alpha':[0, 10**-5, 10**-4, 10**-3, 10**-2, 10**-2, 10**-1, 1, 10, 100, 1000, 10000]
}
gsearch4 = GridSearchCV(estimator = xgb.XGBRegressor( n_estimators=78, 
                                                  max_depth=4,
                                                  min_child_weight=3,
                                                  min_split_loss = 0,
                                                  colsample_bytree= 0.9,
                                                  subsample = 0.85,
                                                  objective = 'reg:squarederror',
                                                  tree_method = 'auto',
                                                  seed=248),
                        param_grid = param_test4, 
                        scoring='explained_variance',
                        n_jobs=-1,
                        cv=5)
gsearch4.fit(X_train,y_train)
gsearch4.best_params_

In [ ]:
# Re-calibrate number of boosting rounds
xgb5 = xgb.XGBRegressor(
    n_estimators=10000,
    max_depth=4,
    min_child_weight=3,
    min_split_loss=0,
    col_sample_bytree = 0.9,
    subsample = 0.85,
    req_alpha = 1000,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    seed=248,
    n_jobs = -1)
cvresult = xgb.cv(xgb5.get_xgb_params(), dtrain, num_boost_round=xgb5.n_estimators, nfold=5,
                      metrics='rmse', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test5 = {
  'n_estimators' : range(50, 1000, 50),
 'learning_rate' : [i/100.0 for i in range(1, 20, 1)]
}
gsearch5 = GridSearchCV(estimator = xgb.XGBRegressor( 
                                                  max_depth=4,
                                                  min_child_weight=3,
                                                  min_split_loss = 0,
                                                  colsample_bytree= 0.9,
                                                  subsample = 0.85,
                                                  req_alpha = 1000,
                                                  objective = 'reg:squarederror',
                                                  tree_method = 'auto',
                                                  seed=248),
                        param_grid = param_test5, 
                        scoring='explained_variance',
                        n_jobs=-1,
                        cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.best_params_

In [ ]:
PlotTuningResults3D(gsearch5, param_test5, 'learning_rate', 'n_estimators')

In [ ]:
param_test4a = {
  'reg_alpha':[0, 10**-5, 10**-4, 10**-3, 10**-2, 10**-2, 10**-1, 1, 10, 100, 1000, 10000]
}
gsearch4a = GridSearchCV(estimator = xgb.XGBRegressor( n_estimators=100, 
                                                      learning_rate = 0.1,
                                                  max_depth=4,
                                                  min_child_weight=3,
                                                  min_split_loss = 0,
                                                  colsample_bytree= 0.9,
                                                  subsample = 0.85,
                                                  objective = 'reg:squarederror',
                                                  tree_method = 'auto',
                                                  seed=248),
                        param_grid = param_test4a, 
                        scoring='explained_variance',
                        n_jobs=-1,
                        cv=5)
gsearch4a.fit(X_train,y_train)
gsearch4a.best_params_

In [ ]:
PlotTuningResults(gsearch4, 'reg_alpha', 'L1 Regularization')

In [ ]:
xgb_final = xgb.XGBRegressor(
    learning_rate = 0.1,
    n_estimators=100,
    max_depth=4,
    min_child_weight=3,
    min_split_loss=0,
    col_sample_bytree = 0.9,
    subsample = .85,
    req_alpha = 100,
    objective= 'reg:squarederror',
    tree_method = 'auto',
    seed=248,
    n_jobs = -1)

In [ ]:
xgb_final.get_xgb_params()
xgb_final.fit(X_train, y_train)
y_pred = xgb_final.predict(X_test)

print(CalculateRMSE(xgb_final, X_test, y_test))
print(explained_variance_score(y_test, y_pred))

In [ ]:
feat_imp = pd.Series(xgb_final.get_booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

## Classification

#### Import data and aggregate datasets

In [ ]:
gene_proteins = pd.read_csv(path+'/data/PAM50_proteins.csv')
clinical = pd.read_csv(path+'/data/clinical_data_breast_cancer.csv')
proteomes = pd.read_csv(path+'/data/77_cancer_proteomes_CPTAC_itraq.csv')

In [ ]:
n = proteomes.index.values
proteomes_trans = proteomes.iloc[:,3:].T
TCGA_ids = pd.DataFrame({"Complete TCGA ID":proteomes_trans.index.values})
proteomes_trans.reset_index(inplace = True)
proteomes_trans.columns.values[0] = 'Complete TCGA ID'

In [ ]:
def ReformatID(ID):
    
    x = ID[3:7]
    y = ID[0:2]
    out = 'TCGA'+'-'+y+'-'+ x
    
    return(out)

In [ ]:
# Data preprocessing
proteomes_trans.iloc[:,0] = [ReformatID(element) for element in proteomes_trans.iloc[:,0]]
proteomes_final = proteomes_trans.dropna(axis = 1)
data = clinical.merge(proteomes_final, on = 'Complete TCGA ID')

objects = data.dtypes[data.dtypes == object].index.values
data_fact = data.copy(deep = True)

for i in objects:
    data_fact.loc[:, i] = pd.factorize(data_fact.loc[:, i])[0]

In [ ]:
# Generate train and test set
seed = np.random.RandomState(248)
y = data_fact['PAM50 mRNA']
X = data_fact.iloc[:, 30:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 248)

#### Boosting

In [ ]:
def PlotTuningResults3D(grid, param_test, parameter1, parameter2):
    m = len(param_test[parameter1])
    n = len(param_test[parameter2])
    
    param_name1 = 'param_' + parameter1
    param_name2 = 'param_' + parameter2
    X = np.reshape(grid.cv_results_[param_name1].data,[n,m])
    Y = np.reshape(grid.cv_results_[param_name2].data,[n,m])
    Z = np.reshape(grid.cv_results_['mean_test_score'],[n,m])
    
    Y = np.array(Y, dtype = float)
    X = np.array(X, dtype = float)
    
    fig = plt.figure(figsize = (12,8))
    ax = fig.gca(projection='3d')

    # Plot the surface.
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    
    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)
    ax.set_xlabel(parameter1, size = 14)
    ax.set_ylabel(parameter2, size = 14)
    ax.set_zlabel('Test accuracy', size = 14)

    plt.show()

In [ ]:
dtrain = xgb.DMatrix(data = X_train, label=list(y_train.values))

In [ ]:
xgb1 = xgb.XGBClassifier(
    n_estimators=10000,
    num_class = 4,
    objective= 'multi:softmax',
    n_jobs = -1,
    seed=248)

cvresult = xgb.cv(xgb1.get_xgb_params(), dtrain, num_boost_round=xgb1.n_estimators, nfold=5,
                      metrics='merror', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
# CV for optimal max depth and child weight
param_test1 = {
 'max_depth':range(1,11,1),
 'min_child_weight':range(0,11,1)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators=43, 
                                                     objective= 'multi:softmax',
                                                     seed=248,
                                                     num_class = 4), 
                        param_grid = param_test1, 
                        scoring='accuracy',
                        n_jobs = -1,
                        cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.best_params_

In [ ]:
PlotTuningResults3D(gsearch1, param_test1, 'max_depth', 'min_child_weight')

In [ ]:
xgb2 = xgb.XGBClassifier(
    n_estimators=10000,
    num_class = 4,
    objective= 'multi:softmax',
    n_jobs = -1,
    seed=248,
    max_depth = 2,
    min_child_weight = 4)

cvresult = xgb.cv(xgb2.get_xgb_params(), dtrain, num_boost_round=xgb2.n_estimators, nfold=5,
                      metrics='merror', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
# CV for optimal min_split_loss
param_test2 = {'min_split_loss':[i/100.0 for i in range(0,2, 1)]}
gsearch2 = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators=6, 
                                                     objective= 'multi:softmax',
                                                     seed=248,
                                                     num_class = 4,
                                                     max_depth = 2,
                                                     min_child_weight = 4), 
                        param_grid = param_test2, 
                        scoring='accuracy',
                        n_jobs = -1,
                        cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.best_params_

In [ ]:
xgb3 = xgb.XGBClassifier(
    n_estimators=10000,
    num_class = 4,
    objective= 'multi:softmax',
    n_jobs = -1,
    seed=248,
    max_depth = 2,
    min_child_weight = 4,
    gamma = 0)

cvresult = xgb.cv(xgb3.get_xgb_params(), dtrain, num_boost_round=xgb3.n_estimators, nfold=5,
                      metrics='merror', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test3 = {
    'subsample':[i/100.0 for i in range(5,105, 5)],
    'colsample_bytree':[i/100.0 for i in range(0,105, 5)]}

gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators=6, 
                                                      max_depth=2,
                                                      min_child_weight=4, 
                                                      gamma=0, 
                                                      objective= 'multi:softmax',
                                                      seed=248,
                                                      num_class = 4), 
                        param_grid = param_test3, 
                        scoring='accuracy',
                        cv=5,
                       n_jobs = -1)
gsearch3.fit(X_train,y_train)
gsearch3.best_params_

In [ ]:
PlotTuningResults3D(gsearch3, param_test3, 'subsample', 'colsample_bytree')

In [ ]:
xgb4 = xgb.XGBClassifier(
    n_estimators=10000,
    num_class = 4,
    objective= 'multi:softmax',
    tree_method = 'auto',
    n_jobs = -1,
    seed=248,
    max_depth = 3,
    min_child_weight = 4,
    gamma = 0,
    colsample_bytree = 0.8,
    subsample = 0.65)

cvresult = xgb.cv(xgb4.get_xgb_params(), dtrain, num_boost_round=xgb4.n_estimators, nfold=5,
                      metrics='merror', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test4 = {
 'reg_alpha':[0, 10**-5, 10**-4, 10**-3, 10**-2, 10**-2, 10**-1, 1, 10, 100, 1000, 10000]
}

gsearch4 = GridSearchCV(estimator = xgb.XGBClassifier(n_estimators=36, 
                                                      max_depth=3,
                                                      min_child_weight=4, 
                                                      gamma=0, 
                                                      colsample_bytree = 0.8,
                                                      subsample = 0.65,
                                                      objective= 'multi:softmax',
                                                      tree_method = 'hist',
                                                      seed=248,
                                                      num_class = 4), 
                        param_grid = param_test4, 
                        scoring='accuracy',
                        cv=5,
                        n_jobs = -1)
gsearch4.fit(X_train,y_train)
gsearch4.best_params_

In [ ]:
def PlotTuningResults_L1(model, parameter, name):
    param_name = 'param_' + parameter
    
    test_scores = model.cv_results_['mean_test_score'][:9]
    param_values = list(model.cv_results_[param_name])[:9]
    
    fig= plt.figure(figsize = (12, 8))
    plt.plot(param_values, test_scores)
    plt.ylim(ymin = 0, ymax = 1)
    plt.ylabel('Test accuracy', size = 20)
    plt.xlabel(name, size = 20)
    plt.yticks(size = 20)
    plt.xticks(param_values, size = 14)
    plt.title('Accuracy vs. ' + name, size = 24)
    plt.tight_layout(pad = 4)

In [ ]:
PlotTuningResults_L1(gsearch4, 'reg_alpha', 'L1 Regularization')

In [ ]:
xgb5 = xgb.XGBClassifier(
    n_estimators=10000,
    num_class = 4,
    objective= 'multi:softmax',
    tree_method = 'auto',
    n_jobs = -1,
    seed=248,
    max_depth = 3,
    min_child_weight = 4,
    gamma = 0,
    colsample_bytree = 0.8,
    subsample = 0.65,
    reg_alpa = 0.1)

cvresult = xgb.cv(xgb5.get_xgb_params(), dtrain, num_boost_round=xgb5.n_estimators, nfold=5,
                      metrics='merror', early_stopping_rounds=50)
cvresult.shape[0]

In [ ]:
param_test5 = {
 'n_estimators' : range(50, 1050, 50),
 'learning_rate' : [i/100.0 for i in range(1, 20, 1)]
}
gsearch5 = GridSearchCV(estimator = xgb.XGBClassifier(max_depth=4,
                                                      min_child_weight=3, 
                                                      gamma=0, 
                                                      subsample=0.8, 
                                                      colsample_bytree=0.65,
                                                      req_alpha = 0.1,
                                                      objective= 'multi:softmax', 
                                                      nthread=4, 
                                                      scale_pos_weight=1,
                                                      seed=248,
                                                      num_class = 4), 
                        param_grid = param_test5, 
                        scoring='accuracy',
                        n_jobs=-1, 
                        cv=5)
gsearch5.fit(X_train,y_train)
gsearch5.best_params_

In [ ]:
PlotTuningResults3D(gsearch5, param_test5, 'learning_rate', 'n_estimators')

In [ ]:
xgb_final = xgb.XGBClassifier(
    learning_rate = 0.15,
    n_estimators=500,
    max_depth=4,
    min_child_weight=3,
    gamma=0,
    col_sample_bytree = 0.8,
    subsample = .65,
    req_alpha = 0.1,
    objective= 'multi:softmax',
    tree_method = 'auto',
    seed=248,
    n_jobs = -1)

In [ ]:
xgb_final.get_xgb_params()
xgb_final.fit(X_train, y_train)
print(metrics.accuracy_score(y_test.values, xgb_final.predict(X_test)))

feat_imp = pd.Series(xgb_final.get_booster().get_fscore()).sort_values(ascending=False)
feat_imp[:30].plot(kind='bar', title='')
plt.ylabel('Feature Importance Score')